# Fit Models Using ALO Optimization on Gisette Data Set

*This notebook builds regularization logistic regression models on the Gisette data set using ALO optimization. It uses several different classes of regularizers.*

## Import Dependencies

In [1]:
import peak_engines
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')

## Run Cross Validation

In [2]:
def compute_cv(model, to_hyper_str, X, y):
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = []
    for train_indexes, test_indexes in cv.split(X):
        X_train = X[train_indexes, :]
        y_train = y[train_indexes]
        X_test = X[test_indexes, :]
        y_test = y[test_indexes]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        model.fit(X_train, y_train)
        pred = model.predict_proba(X_test)
        score = -log_loss(y_test, pred)
        print(to_hyper_str(model), score)
        scores.append(score)
    print("mean", np.mean(scores))

## Load Gisette Data Set

In [3]:
url_X_train = 'https://archive.ics.uci.edu/ml/machine-learning-databases/gisette/GISETTE/gisette_train.data'
url_y_train = 'https://archive.ics.uci.edu/ml/machine-learning-databases/gisette/GISETTE/gisette_train.labels'
url_X_valid = 'https://archive.ics.uci.edu/ml/machine-learning-databases/gisette/GISETTE/gisette_valid.data'
url_y_valid = 'https://archive.ics.uci.edu/ml/machine-learning-databases/gisette/gisette_valid.labels'
X_train = pd.read_csv(url_X_train, delim_whitespace=True, header=None).iloc[:,:].values
y_train = pd.read_csv(url_y_train, delim_whitespace=True, header=None).iloc[:,0].values
X_valid = pd.read_csv(url_X_valid, delim_whitespace=True, header=None).iloc[:,:].values
y_valid = pd.read_csv(url_y_valid, delim_whitespace=True, header=None).iloc[:,0].values
X = np.vstack((X_train, X_valid))
y = np.concatenate((y_train, y_valid))

## Build Models

### Grid Search

In [4]:
model = LogisticRegressionCV(random_state=0, scoring='neg_log_loss')
to_hyper_str = lambda model: str(1/(2*model.C_[0]))
compute_cv(model, to_hyper_str, X, y)

10.772173450159421 -0.07141330300734189
10.772173450159421 -0.05860679105760188
10.772173450159421 -0.06974745365248325
10.772173450159421 -0.07259035146418859
10.772173450159421 -0.0669355350350212
mean -0.06785868684332737


### Ridge Regularization

In [5]:
model = peak_engines.LogisticRegressionModel()
to_hyper_str = lambda model: str(1/(2*model.C_[0]))
compute_cv(model, to_hyper_str, X, y)

19.125168179845378 -0.07091119288281082
18.657674666781162 -0.05869256428746778
18.264972939868283 -0.06970706513727046
16.48213583423184 -0.07143037658300683
14.35124635778496 -0.06667426040993736
mean -0.06748309186009865


### Bridge Regularization

In [6]:
model = peak_engines.LogisticRegressionModel(penalty='bridge')
to_hyper_str = lambda model: str([model.hyperparameters_[0]**2, 1 + model.hyperparameters_[1]**2])
compute_cv(model, to_hyper_str, X, y)

[17.2634409704122, 1.947447741452002] -0.06979763706313755
[16.787588530291675, 1.9435606478793317] -0.0580893310494714
[12.952675285512477, 1.8068898578625712] -0.06667017379085097
[10.543638008653765, 1.7527586105928714] -0.06772661499352636
[10.771891713285207, 1.828397324061291] -0.06387877783012405
mean -0.06523250694542207
